[![Open In Colab](https://colab.research.googleusercontent.com/assets/colab-badge.svg)](https://colab.research.google.com)

Si tu repo está en GitHub, usa este enlace editando USER/REPO/BRANCH:
[Open in Colab (GitHub)](https://colab.research.google.com/github/USER/REPO/blob/BRANCH/professor/pandas_v2/05_eda_groupby_multiindex.ipynb)



# 05 - EDA con pandas y GroupBy (incluye MultiIndex)

Objetivos:
- Cargar los datos limpios y explorar con `DataFrame.info`, `DataFrame.describe`.
- Usar `DataFrame.groupby` (una y varias claves) con agregaciones nombradas (`.agg`).
- Trabajar con `MultiIndex`: `reset_index`, `unstack`, `xs`, `swaplevel`, `sort_index`.
- Tabulados rápidos: `value_counts`, `crosstab`, `pivot_table` (con `margins`, `fill_value`).
- Agrupar por tiempo con `pd.Grouper` (si hay columna `fecha`).


In [5]:
import pandas as pd
from pathlib import Path

path = Path("data/clean/dataset_limpio.csv")
print("Cargando:", path.as_posix())
df = pd.read_csv(path)
print("\nInfo rápida:")
print(df.info())
print("\nDescribe:")
print(df.describe(include="all"))
print("\nPrimeras filas:")
print(df.head())


Cargando: data/clean/dataset_limpio.csv

Info rápida:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   venta_id         6 non-null      int64  
 1   cliente_id       5 non-null      float64
 2   fecha            1 non-null      object 
 3   monto            3 non-null      float64
 4   categoria_final  6 non-null      object 
 5   canal            6 non-null      object 
 6   nombre           3 non-null      object 
 7   pais             3 non-null      object 
dtypes: float64(2), int64(1), object(5)
memory usage: 516.0+ bytes
None

Describe:
        venta_id  cliente_id                      fecha       monto  \
count   6.000000    5.000000                          1    3.000000   
unique       NaN         NaN                          1         NaN   
top          NaN         NaN  2024-01-01 00:00:00+00:00         NaN   
freq         NaN   

## GroupBy y agregaciones nombradas

Funciones clave en la celda de abajo:
- `DataFrame.groupby([...])` para agrupar por una o varias columnas.
- `.agg(...)` con agregaciones nombradas: `total=("monto","sum")`, etc.
- `as_index` controla si las claves se vuelven índice; `reset_index()` recupera columnas.
- Tips: `sort_values`, `sort_index`, `dropna`, `observed` (categorías).


In [6]:
print("GroupBy por categoria_final:")
print(
    df.groupby("categoria_final").agg(
        total=("monto", "sum"),
        promedio=("monto", "mean"),
        conteo=("monto", "size"),
    )
)

print("\nGroupBy múltiple (categoria_final, nombre) y MultiIndex:")
res = df.groupby(["categoria_final", "nombre"]).agg(
    total=("monto", "sum"),
    trans=("monto", "size"),
)
print(res)

print("\n`unstack` para formato de reporte:")
print(res.unstack(0))


GroupBy por categoria_final:
                 total  promedio  conteo
categoria_final                         
A                100.0    100.00       2
B                500.5    250.25       3
DESCONOCIDO        0.0       NaN       1

GroupBy múltiple (categoria_final, nombre) y MultiIndex:
                        total  trans
categoria_final nombre              
A               Ana     100.0      1
B               Luis    200.5      2

`unstack` para formato de reporte:
                 total        trans     
categoria_final      A      B     A    B
nombre                                  
Ana              100.0    NaN   1.0  NaN
Luis               NaN  200.5   NaN  2.0


## GroupBy con `as_index=False` y más métricas

- `as_index=False` deja las claves como columnas.
- Agregaciones múltiples en diferentes columnas usando `.agg` con dict/tuplas.
- `sort_values` para ordenar el resultado de forma legible.


In [7]:
print("GroupBy con as_index=False y múltiples agregaciones:")
res2 = (df.groupby("categoria_final", as_index=False)
          .agg(total=("monto","sum"), promedio=("monto","mean"), distintos_nombres=("nombre","nunique"))
          .sort_values("total", ascending=False))
print(res2)



GroupBy con as_index=False y múltiples agregaciones:
  categoria_final  total  promedio  distintos_nombres
1               B  500.5    250.25                  1
0               A  100.0    100.00                  1
2     DESCONOCIDO    0.0       NaN                  0


## Operaciones útiles en MultiIndex

Funciones mostradas abajo:
- `xs(key, level=...)` para cortar una sección del índice por nivel.
- `swaplevel()` para reordenar niveles; `sort_index(level=...)` para ordenar.
- `unstack()` ya mostrado; `stack()` para el reverso.


In [8]:
print("Extraemos con xs() el nivel categoria_final='A':")
try:
    print(res.xs('A', level=0))
except Exception as e:
    print("Nota: asegúrate de ejecutar la celda de groupby múltiple primero.")

print("\nReordenamos niveles con swaplevel() y ordenamos índice:")
try:
    swapped = res.swaplevel(0,1).sort_index(level=0)
    print(swapped.head())
except Exception as e:
    pass



Extraemos con xs() el nivel categoria_final='A':
        total  trans
nombre              
Ana     100.0      1

Reordenamos niveles con swaplevel() y ordenamos índice:
                        total  trans
nombre categoria_final              
Ana    A                100.0      1
Luis   B                200.5      2


## Frecuencias y tablas resumidas

- Frecuencias: `value_counts(normalize=True)`.
- `crosstab` para contingencias.
- `pivot_table` para resúmenes en formato wide (agregación incluida).


In [9]:
print("Frecuencia de nombres (normalizada):")
print(df["nombre"].value_counts(normalize=True))

print("\nCrosstab categoria_final x nombre (conteos):")
print(pd.crosstab(df["categoria_final"], df["nombre"]))

print("\nPivot table de montos por categoria_final y nombre (suma):")
print(pd.pivot_table(df, index="categoria_final", columns="nombre", values="monto", aggfunc="sum"))


Frecuencia de nombres (normalizada):
nombre
Luis    0.666667
Ana     0.333333
Name: proportion, dtype: float64

Crosstab categoria_final x nombre (conteos):
nombre           Ana  Luis
categoria_final           
A                  1     0
B                  0     2

Pivot table de montos por categoria_final y nombre (suma):
nombre             Ana   Luis
categoria_final              
A                100.0    NaN
B                  NaN  200.5
